여러 모델을 저장하고 불러오는 것은 이전에 학습했던 모델들을 재사용하는데 도움이 된다
<br><br>

## 개요
GAN이나 시퀀스-투-시퀀스(sequence-to-sequence model), 앙상블 모델(ensemble of models)과 같이 여러 ```torch.nn.Modules``` 로 구성된 모델을 저장할 때는 각 모델의 state_dict와 해당 옵티마이저의 dictionary를 저장해야한다. 또한, 학습을 재개하는데 필요한 다른 항목들을 dictionary에 추가할 수 있다. 모델들을 불러올 때에는, 먼저 모델들과 옵티마이저를 초기화하고, ```torch.load()```를 사용하여 dictionary를 불러온다. 이후 원하는대로 저장한 항목들을 dictionary에 조회하여 접근할 수 있다. 

## 단계
1. 데이터를 불러올 때 필요한 라이브러리들 불러오기
2. 신경망을 구성하고 초기화하기
3. 옵티마이저 초기화하기
4. 여러 모델들 저장하기
5. 여러 모델들 불러오기


### 1. 데이터를 불러올 때 필요한 라이브러리들 불러오기

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

### 2. 신경망을 구성하고 초기화하기


In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
netA = Net()
netB = Net()

### 3. 옵티마이저 초기화하기
생성한 모델들을 각각에 모멘텀을 갖는 SGD를 사용

In [3]:
optimizerA = optim.SGD(netA.parameters(), lr=0.001, momentum=0.9)
optimizerB = optim.SGD(netB.parameters(), lr=0.001, momentum=0.9)

### 4. 여러 모델들 저장하기
관련된 모든 정보들을 모아서 사전을 구성

In [4]:
# 저장할 경로 지정
PATH = "model.pt"

torch.save({
    'modelA_state_dict':netA.state_dict(),
    'modelB_state_dict':netB.state_dict(),
    'optimizerA_state_dict':optimizerA.state_dict(),
    'optimizerB_state_dict':optimizerB.state_dict(),
    }, PATH)

### 5. 여러 모델들 불러오기
먼저 모델과 옵티마이저를 초기화한 뒤, dictionary를 불러와야한다.

In [5]:
modelA = Net()
modelB = Net()
optimModelA = optim.SGD(modelA.parameters(), lr=0.001, momentum=0.9)
optimModelB = optim.SGD(modelB.parameters(), lr=0.001, momentum=0.9)

checkpoint = torch.load(PATH)
modelA.load_state_dict(checkpoint['modelA_state_dict'])
modelB.load_state_dict(checkpoint['modelB_state_dict'])
optimizerA.load_state_dict(checkpoint['optimizerA_state_dict'])
optimizerB.load_state_dict(checkpoint['optimizerB_state_dict'])

modelA.eval()
modelB.eval()
# -- 또는 --
modelA.train()
modelB.train()

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

inference를 실행하기 전에 ```model.eval()```을 호출하여 드롭아웃(dropout)과 배치 정규화 층(batch normalization layer)을 evaluation mode로 바꿔야한다. <br>
만약 학습을 계속하길 원한다면 ```model.train()```을 호출하여 이 층(layer)들이 학습 모드인지 확인해야한다.